In [1]:
import pandas


In [7]:
institution_data= pandas.read_csv('/home/shubhamjain/devel/contifydeploy/contify-banking/notebooks/data-challenge-data-master/2012_to_2014_institutions_data.csv')
loans_data= pandas.read_csv('/home/shubhamjain/devel/contifydeploy/contify-banking/notebooks/data-challenge-data-master/2012_to_2014_loans_data.csv')

/home/shubhamjain/.pyenv/versions/banking/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4,5,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
institution_data.describe()


,As_of_Year,Agency_Code,Assets_000_Panel
count,21655.000000,21655.000000,2.165500e+04
mean,2012.984392,3.785592,4.976024e+06
std,0.817139,1.944049,6.140364e+07
min,2012.000000,1.000000,0.000000e+00
25%,2012.000000,3.000000,8.380000e+04
50%,2013.000000,3.000000,1.941060e+05
75%,2014.000000,5.000000,4.908065e+05
max,2014.000000,9.000000,1.945467e+09


In [6]:
institution_data.columns

Index([u'As_of_Year', u'Respondent_ID', u'Agency_Code', u'Respondent_Name_TS',
       u'Respondent_City_TS', u'Respondent_State_TS', u'Respondent_ZIP_Code',
       u'Parent_Name_TS', u'Parent_City_TS', u'Parent_State_TS',
       u'Parent_ZIP_Code', u'Assets_000_Panel'],
      dtype='object')

In [8]:
loans_data.describe()


,Agency_Code,As_of_Year,Loan_Amount_000,Sequence_Number,State_Code,Conforming_Limit_000
count,1321158.000000,1321158.000000,1321158.000000,1321158.000000,1321158.000000,1320321.000000
mean,6.852763,2012.797469,290.233111,162353.932230,38.513506,532.910087
std,2.486604,0.775239,965.779154,334661.258178,15.257442,86.907939
min,1.000000,2012.000000,1.000000,0.000000,10.000000,417.000000
25%,7.000000,2012.000000,153.000000,2322.000000,24.000000,458.000000
50%,7.000000,2013.000000,235.000000,16128.000000,51.000000,535.000000
75%,9.000000,2013.000000,347.000000,140628.000000,51.000000,625.000000
max,9.000000,2014.000000,99625.000000,3076577.000000,54.000000,625.000000


In [10]:
 grouped = loans_data.groupby('State_Code')

In [14]:
statewize = grouped.describe()


In [26]:
statewize

Agency_Code     As_of_Year  Loan_Amount_000  \
State_Code                                                        
10         count   61706.000000   61706.000000     61706.000000   
           mean        7.099472    2012.817295       208.151379   
           std         2.303211       0.775842       114.197591   
           min         1.000000    2012.000000         1.000000   
           25%         7.000000    2012.000000       140.000000   
           50%         7.000000    2013.000000       192.000000   
           75%         9.000000    2013.000000       256.000000   
           max         9.000000    2014.000000      4000.000000   
11         count   49461.000000   49461.000000     49461.000000   
           mean        6.986656    2012.809951       414.584036   
           std         2.404837       0.782434       285.075430   
           min         1.000000    2012.000000         4.000000   
           25%         7.000000    2012.000000       248.000000   
           50%         7.000000    2013.000000       364.000000   
           75%         9.000000    2013.000000       520.000000   
           max         9.000000    2014.000000      8000.000000   
24         count  452637.000000  452637.000000    452637.000000   
           mean        6.910277    2012.784730       278.306349   
           std         2.440085       0.774037       169.860550   
           min         1.000000    2012.000000         1.000000   
           25%         7.000000    2012.000000       170.000000   
           50%         7.000000    2013.000000       248.000000   
           75%         9.000000    2013.000000       350.000000   
           max         9.000000    2014.000000      4720.000000   
51         count  679703.000000  679703.000000    679703.000000   
           mean        6.885643    2012.793182       304.950097   
           std         2.449575       0.773373      1164.093277   
           min         1.000000    2012.000000         1.000000   
           25%         7.000000    2012.000000       155.000000   
           50%         7.000000    2013.000000       238.000000   
           75%         9.000000    2013.000000       356.000000   
           max         9.000000    2014.000000     99625.000000   
54         count   77651.000000   77651.000000     77651.000000   
           mean        5.948359    2012.885552       216.953033   
           std         3.027491       0.787408      1937.582771   
           min         1.000000    2012.000000         1.000000   
           25%         3.000000    2012.000000        79.000000   
           50%         7.000000    2013.000000       125.000000   
           75%         9.000000    2014.000000       183.000000   
           max         9.000000    2014.000000     99306.000000   

                  Sequence_Number  State_Code  Conforming_Limit_000  
State_Code                                                           
10         count     61706.000000       61706          61593.000000  
           mean     193418.092973          10            417.000000  
           std      360949.757792           0              0.000000  
           min           1.000000          10            417.000000  
           25%        3202.000000          10            417.000000  
           50%       26028.500000          10            417.000000  
           75%      192152.000000          10            417.000000  
           max     3070284.000000          10            417.000000  
11         count     49461.000000       49461          49451.000000  
           mean     150114.599947          11            625.000000  
           std      318415.136283           0              0.000000  
           min           1.000000          11            625.000000  
           25%        2519.000000          11            625.000000  
           50%       10814.000000          11            625.000000  
           75%      128384.000000          11            625.000000  
           max 

In [32]:
import matplotlib.pyplot as plt
pandas.options.display.mpl_style = 'default'
institution_data.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x11053c50>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x74a3d10>]], dtype=object)